In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import gc

#read images
pimages = glob("C:/Users/Du/Documents/ref-matte/train/mask/*")
#images = glob("../imgs/*")
print(f"Images: {len(pimages)}")


images = pimages[:4999]
print(f"Images: {len(images)}")
del pimages
gc.collect()

if not os.path.exists("results/"):
    os.makedirs("results")

#separate images by name and store in names dictionary
dic = dict()
print("resolving names...")
for x in tqdm(images, total = len(images)):
        """ Extracting the name """
        name = x.split("\\")[-1] #windows format
        nameVec = []
        nameVec.append(x)
        nameKey = name.split("_")[2]
        if nameKey not in dic:
            dic[nameKey] = []
        dic[nameKey].append(nameVec[0])

#for each key read images and put in images dic
image = dict()
print("resolving images...")
for key in tqdm(dic, total = len(dic)):
      image[key] = []
      for value in dic[key]:    
          image[key].append(cv2.imread(value, cv2.IMREAD_COLOR))

#for each key merge images inside and save in path
del dic
gc.collect

imagelist = [[0 for x in range(4)] for y in range(len(image.keys()))]
print(len(imagelist))
print(len(imagelist[0]))
i = 0
print("blending images...")
for key in tqdm(image, total = len(image)):
    for j in range(len(image[key])):
        lenimagekey = len(image[key])
        print("lenimagekey ", lenimagekey)
        if j == 0:
            imagelist[i] = image[key][j]
        else:
            imagelist[i] = cv2.addWeighted(imagelist[i], 1.0, image[key][j], 1.0, 0)
    name = "".join(["rim_train_", key,".jpg"]) 
    save_image_path = os.path.join("results", name)
    cv2.imwrite(save_image_path, imagelist[i])
    del imagelist[i]
    del image[key]
    gc.collect()
    i += 1
